In [26]:
from selenium import webdriver
import pandas as pd
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import time
import re

path = 'C:/Users/sunny/test/geckodriver.exe'
driver = webdriver.Firefox(executable_path=path)
driver.get('http://www.health.kr/searchDrug/search_detail.asp')

driver.find_element_by_id("anchor_form_info").click()
driver.find_element_by_id("form_info_TB").click()
driver.find_element_by_id("btn_detail_search").click()
time.sleep(10)
driver.find_element_by_id("anchor_proy_more").click()
time.sleep(50)
###100줄 보는 코드
driver.find_element_by_xpath('//*[@id="div_lines"]/span').click()
time.sleep(5)
driver.find_element_by_id('line_li_100').click()

In [31]:
img_re = re.compile('src=\"[^"]+\"')

pills = []
prev_page = '0'
while True:
    html=driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    current_page = soup.find("a", {"class":"current", "target":"_self"}).text
    
    ###page 완전히 넘겨질 때까지 대기
    sleep = 0
    while prev_page == current_page:
        if sleep > 200:
            break
        time.sleep(2)
        sleep += 1
        html=driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        current_page = soup.find("a", {"class":"current", "target":"_self"}).text
    if sleep > 200: break
        
    ###pills에 알약별 이미지    
    table = soup.find('table', {'class':'tab3_cont list_basic bt0'})
    
    for tr in table.find_all('tr'):
        ele = list(tr.find_all('td'))
        if len(ele) > 0:
            img = img_re.findall(str(ele[0]))[0][5:-1]
            name = ele[1].text
            component = ele[2].text
            effect = ele[3].text
            company = ele[4].text
            
            pills.append([img, name, component, effect, company])
    
    ####################식별 마크 정보 추가####################
    # 아직 에러 남 : for문 안쓰고 사진 하나씩은 뜨는데, for 문 안에서 다음 사진으로 넘어갈 때 에러 발생
    for i in range(1,len(table.find_all('tr'))):
        if pills[i-1][0].startswith("/images"):
            continue
        driver.find_element_by_xpath('//*[@id="tbl_pro"]/tbody/tr['+str(i+1)+']/td[1]').click()
        time.sleep(5)
        html=driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        front = soup.find('td', {'id':'idfy_front'})
        print(front)
        back = soup.find('td', {'id':'idfy_back'})
        print(back)
        
        """
        front = driver.find_element_by_id("idfy_front").text
        back = driver.find_element_by_id("idfy_back").text
        driver.find_element_by_xpath('//*[@id="pop_more"]/a[1]').click()
        time.sleep(5)
        pills[i-1].append(front)
        pills[i-1].append(back)"""
       
    print("page {} done.".format(current_page))
    prev_page = current_page
    driver.find_element_by_class_name("right.ico").click()

<td id="idfy_front">Y
<br/>50</td>
<td id="idfy_back"></td>
<td id="idfy_front">ID 50</td>
<td id="idfy_back"></td>
<td id="idfy_front">RY</td>
<td id="idfy_back">GM</td>
<td id="idfy_front">G
<br/>50</td>
<td id="idfy_back"><img align="absmiddle" height="20" name="img1" src="http://www.pharm.or.kr/images/sb_photo/mark/r0426.gif" width="20"/></td>
<td id="idfy_front"><img align="absmiddle" height="20" name="img1" src="http://www.pharm.or.kr/images/sb_photo/mark/r0608.gif" width="20"/></td>
<td id="idfy_back">GN</td>
<td id="idfy_front"><img align="absmiddle" class="linev" name="img1" src="http://www.pharm.or.kr/images/line_v.gif"/></td>
<td id="idfy_back">HC
<br/>803</td>
<td id="idfy_front">J<img align="absmiddle" class="linev" name="img1" src="http://www.pharm.or.kr/images/line_v.gif"/>Y</td>
<td id="idfy_back">M</td>
<td id="idfy_front">BT</td>
<td id="idfy_back">50</td>


KeyboardInterrupt: 

In [18]:
pills

[['https://www.pharm.or.kr:442/images/sb_photo/big3/A11AOOOOO895502.jpg',
  '가나릴정',
  'Itopride Hydrochloride 50mg ',
  '효능 / 효과[허가사항변경(2011년 재평가), 의약품관리과-6843, 2012.12.31)](정제)  기능성소화불량으로 인한 소화기증상(복부팽만, 상복부통, 식욕부진, 속쓰림, 구역, 구토)',
  '영풍제약',
  ['Y\n50'],
  ''],
 ['https://www.pharm.or.kr:442/images/sb_photo/big3/A11AKP08G301601.jpg',
  '가나메드정',
  'Itopride Hydrochloride 50mg ',
  '효능 / 효과[허가사항변경(2011년 재평가), 의약품관리과-6843, 2012.12.31)](정제)  기능성소화불량으로 인한 소화기증상(복부팽만, 상복부통, 식욕부진, 속쓰림, 구역, 구토)',
  '일동제약',
  ['ID 50'],
  ''],
 ['https://www.pharm.or.kr:442/images/sb_photo/big3/A11AKP08A042601.jpg',
  '가나모티정',
  'Mosapride Citrate Hydrate 5.29mg',
  '효능 / 효과[허가사항변경(2011년 재평가), 의약품관리과-6843, 2012.12.31)](정제)(산제)기능성소화불량으로 인한 소화기증상(속쓰림, 구역, 구토)',
  '이연제약',
  ['RY'],
  'GM'],
 ['/images/img_empty3.jpg',
  '가나젠정',
  'Itopride Hydrochloride 50mg ',
  '효능 / 효과기능성소화불량으로 인한 소화기증상(복부팽만, 상복부통, 식욕부진, 속쓰림, 구역, 구토)',
  '테라젠이텍스'],
 ['https://www.pharm.or.kr:442/images/sb_photo/big3/A11A0210A065702.jpg',
  '가나칸정